In [10]:
import pandas as pd
import numpy as np
from datetime import datetime
from matplotlib import pyplot as plt
%matplotlib inline

In [11]:
df_str_dh = pd.read_csv(filepath_or_buffer='../statistic_results/statistic-#17.csv', header=0, sep=',', dtype=np.longlong)
df_str_dh['FULL_DURATION'] = df_str_dh['DURATION_END']-df_str_dh['DURATION_START']
df_str_dh['KEY_AGREEMENT_DURATION'] = df_str_dh['DURATION_END']-df_str_dh['KEY_AGREEMENT_START']
df_str_dh['INITIALIZATION_DURATION'] = df_str_dh['FULL_DURATION']-df_str_dh['KEY_AGREEMENT_DURATION']

print("FULL DURATION: {:.2f}ms".format(df_str_dh['FULL_DURATION'].mean()/1000_000))
print("KEY AGREEMENT DURATION: {:.2f}ms".format(df_str_dh['KEY_AGREEMENT_DURATION'].mean()/1000_000))
print("INITIALIZATION DURATION: {:.2f}ms".format(df_str_dh['INITIALIZATION_DURATION'].mean()/1000_000))

FULL DURATION: 715.55ms
KEY AGREEMENT DURATION: 696.00ms
INITIALIZATION DURATION: 19.55ms
